## NVflare Quickstart

NVflare Quick start docs  
https://nvflare.readthedocs.io/en/2.5.1/quickstart.html  

NVflare git hub  
https://github.com/NVIDIA/NVFlare

NVflare를 설치하기 위하여, Python virtual environment 또는 docker를 활용하여야 합니다.  
따라서 본 ipynb 문서의 셀에서 코드를 실행하지 마시고, 원하는 환경의 cmd(terminal)에 명령어를 복사하여 실행하세요  


NVflare는 Python3.8 ~ 3.10까지를 지원합니다. 따라서 venv 환경 구성시 파이썬 버전을 지정하여 주어야 합니다

In [ ]:
# FLARE  설치

sudo apt update
sudo apt-get install python3-venv
# apt를 업데이트하고, venv 패키지 설치

python3.8 -m venv nvflare-env
# 파이썬 3.11이후로 지원하지 않으므로 3.8로 변경하여 가상환경 생성

source nvflare-env/bin/activate
# venv의 source activate(가상환경 activate)

python3 -m pip install -U pip
python3 -m pip install -U setuptools
# venv 가상환경 내에 pip 및 setuptools 설치

python3 -m pip install nvflare
# nvflare 설치

git clone https://github.com/NVIDIA/NVFlare.git
# github 복제

cd NVFlare
# git에서 받은 NVflare 디렉토리로 이동

git switch 2.5
# 현재 가장 최신 안정화 릴리즈인 2.5 버전으로 스위치

## NVflare Simulator

NVflare Simulator  
https://nvflare.readthedocs.io/en/2.5.1/user_guide/nvflare_cli/fl_simulator.html#  

시뮬레이터는 NVflare를 테스트하는데 가장 간단한 방법인 경량, 자동화 시뮬레이션 입니다.
사용자는 시뮬레이터를 이용하여 NVflare를 실제로 프로비저닝 하지 않고도 테스트가 가능합니다.  
실제로 분리된 서버-클라이언트 준비, 환경 구성, 네트워크연결, 사용자등록, Job 배포 등의 과정이 없으므로 간편하게 실험하고 오류를 탐색할 수 있습니다.  

하나의 프로세스(추가 가능)내에서 서버와 여러 클라이언트를 구성하고 사용합니다.  

In [ ]:
# FL 시뮬레이터 실행

nvflare simulator -h
# 시뮬레이터 인수확인 명령어(--help)
# 실행하였을 때 usage 및 사용가능한 인자(arguments)등이 출력되면 정상적으로 설치된 것입니다

# hello-pt 예제 실행
# git repo 복사와 브랜치(2.5)선택은 이미 진행된 것으로 가정 + 가상환경 activate도

mkdir simulator-example
cp -rf NVFlare/examples/hello-world/hello-pt simulator-example/
# 새로운 예제 테스트 디렉토리를 생성하고 hello-pt 예제를 복사하여 사용
# 원하는 위치에 디렉토리를 생성하세요 / git클론으로 생성된 NVflare 디렉토리 바깥을 권장

python3 -m pip install -r simulator-example/hello-pt/requirements.txt
# requirements 설치를 통하여 종속성 설치

cd simulator-example/hello-pt
python3 fedavg_script_runner_pt.py
# fedavg_script_runner_pt를 실행하여 cifar10 연합학습 예제를 실행 가능
# Server, site1, site2가 생성되어 cifar10 데이터로 연합학습을 자동으로 진행하는 것을 볼 수 있음

sudo apt-get install tree
tree -L 4 /tmp/nvflare/jobs/workdir
# tree를 설치하고 사용하면 tmp/nvflare에 생성된 작업 디렉토리의 구조를 확인가능

아래는 위에서 실행한 fedavg_script_runner_pt.py의 내용입니다.

In [ ]:
# fedavg_script_runner_pt.py

from src.simple_network import SimpleNetwork
from nvflare.app_opt.pt.job_config.fed_avg import FedAvgJob
from nvflare.job_config.script_runner import ScriptRunner

if __name__ == "__main__":
    
    n_clients = 2
    # 참여 클라이언트 수
    
    num_rounds = 2
    # 실행할 round 수
    
    train_script = "src/hello-pt_cifar10_fl.py"
    # 학습할 스크립트 예제 내용. 클라이언트의 연합학습 동작이 들어있습니다.
    
    job = FedAvgJob(
        # NVflare에서는 job을 통해 서버-클라이언트간 작업을 정의합니다.
        name="hello-pt_cifar10_fedavg", n_clients=n_clients, num_rounds=num_rounds, initial_model=SimpleNetwork()
        # 작업의 이름, 사용할 클라이언트 수, 반복할 round 수, 사용할 기본 모델 지정
    )

    for i in range(n_clients):
        # 참여할 클라이언트의 수 만큼 excutor를 실행시킵니다.
        executor = ScriptRunner(
            script=train_script, script_args=""
            # f"--batch_size 32 --data_path /tmp/data/site-{i}"
            # executor에 주어질 스크립트는 학습 스크립트이고, 추가적인 인자는 주지 않겠다는 의미입니다.
            # (hello-pt_cifar10_fl.py 파일에서 이미 클라이언트의 하이퍼파라미터를 지정하였기 때문에)
            # 주어질 하이퍼 파라미터 예시를 보면, 학습할 데이터의 위치와 batch사이즈 등을 지정할 수 있음을 알 수 있습니다.
        )
        job.to(executor, f"site-{i+1}")
        # excutor를 실행시켜 클라이언트에게 작업을 전달합니다.

    # job.export_job("/tmp/nvflare/jobs/job_config")
    # 아래 시뮬레이터가 아닌 위 export_job을 사용하면,
    # 실제 환경의 작업처럼 사용자가 정의한 job(job_config)을 활용하여 서버 > 클라이언트로 작업을 배포 할 수 있습니다.
    
    job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="1")
    # export_job이 아닌 simulator_run은 하나 또는 복수 프로세스 내에서 여러 서버와 클라이언트의 연합학습을 자동으로 실행하고 결과를 반환합니다
    # 이것이 simulator 모드입니다.

아래는 위 fedavg_script_runner_pt.py에서 호출된 src/hello-pt_cifar10_fl.py의 내용입니다.  
클라이언트의 동작, 즉 클라이언트의 학습 과정과 flare 서버로부터 초기모델을 전달받고 다시 업로드하는 일련의 과정을 확인 가능합니다.  

In [ ]:
import os
import torch
from simple_network import SimpleNetwork
# simple_network.py 파일에 사용할 모델을 따로 정의하고 호출
from torch import nn
from torch.optim import SGD
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor
import nvflare.client as flare
from nvflare.client.tracking import SummaryWriter
# 필요 패키지 import

#-----------------------------------------------------------------------------------------------------------
DATASET_PATH = "/tmp/nvflare/data"
# 데이터셋 경로 지정
# job을 통해 생성된 workspace 내부 data 경로

#-----------------------------------------------------------------------------------------------------------
# 아래는 클라이언트가 동작할 메인 함수 작성입니다
def main():
    batch_size = 4
    epochs = 5
    lr = 0.01
    # 배치사이즈, Epoch, Learning rate 지정
    model = SimpleNetwork()
    # SimpleNetwork를 호출하여 model로 저장

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # 사용할 GPU 지정, 없으면 CPU
    
    loss = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)
    # 손실값과 최적화 함수를 정의한다
    
    transforms = Compose(
        [
            ToTensor(),
            Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
    # 딥러닝 학습을 위하여 데이터를 텐서 행렬화하고 정규화함

    flare.init()
    sys_info = flare.system_info()
    client_name = sys_info["site_name"]
    # Flare 시스템 초기화 후 시스템 정보 제공하여 시작, 클라이언트 이름은 sys_info에 포함된 것으로

    train_dataset = CIFAR10(
        root=os.path.join(DATASET_PATH, client_name), transform=transforms, download=True, train=True
    )
    # 학습 데이터를 정의하는 부분, 학습데이터는 CIFAR10 이미지이며, 위에서 설정한 데이터 경로와 클라이언트별 이름을 join
    # 데이터에 대해 transform하고, 데이터가 없다면 다운로드 받을 것이며, 학습용 데이터셋임을 명시
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # 학습데이터 로더를 데이터로더 함수를 통하여 정의, 학습데이터셋, 배치사이즈, 데이터 섞기 활성화
    
# 여기까지는 딥러닝 학습을 진행하기 위해 필요한 데이터, 데이터의 분할(배치사이즈), 데이터 로더, 학습 횟수와 학습 률,
# 학습에 사용할 장비 CPU/GPU, 손실값과 최적화함수, 사용할 딥러닝 모델, NVflare에게 알려줄 현재 작동되는 클라이언트 를 정의한 부분입니다
#-----------------------------------------------------------------------------------------------------------
    summary_writer = SummaryWriter()
    while flare.is_running():
        input_model = flare.receive()
        # 모델을 플레어로부터 받는다
        
        print(f"current_round={input_model.current_round}")
        # 현재 라운드를 출력하는 부분
        
        model.load_state_dict(input_model.params)
        model.to(device)
        # 모델은 load state dict를 사용하여 플레어로부터 받은 모델의 파라미터를 가져와 로드한다
        # 모델의 파라미터를 위에서 지정한 학습 장치 GPU 또는 CPU에 올린다

        steps = epochs * len(train_loader)
        # step 수라는 것을 정의하고 이는 epoch * 배치사이즈로 한다
        
# flare 서버로부터 사용할 모델의 파라미터(글로벌 모델 파라미터)를 받아오고, 이것을 가지고있는 모델 model에 적용하여 글로벌 모델과 같게 만든다
#-----------------------------------------------------------------------------------------------------------
        for epoch in range(epochs):
        # 정의된 Epoch 학습 횟수만큼 반복(학습)하는데,
            
            running_loss = 0.0
            # 처음의 loss를 0으로 규정하고
            
            for i, batch in enumerate(train_loader):
            # 학습 데이터로더의 현재 순서(로더중 몇번쨰인지)와 batch(전체 데이터의 분할조각)를 받아서
                
                images, labels = batch[0].to(device), batch[1].to(device)
                # 전체 batch중 앞과 뒤를 나눠 학습용과 정답레이블로 사용하고
                
                optimizer.zero_grad()
                predictions = model(images)
                cost = loss(predictions, labels)
                # 학습용 데이터를 주고 예측을 생성하여 예측과 정답을 비교하고 차이값 만큼을 loss > cost로 저장한다
                
                cost.backward()
                optimizer.step()
                # 역전파 후 최적화 함수의 파라미터 최적화 진행

                running_loss += cost.cpu().detach().numpy() / images.size()[0]
                # 한 epoch의 loss 값은 cost값의 데이터인 image의 크기로 나눈 것으로 함
                
# 여기까지 모델 학습이 진행되는 부분, 기존 딥러닝 모델의 학습 함수와 기능이 같지만, NVflare의 클라이언트 관리 기능이 동작할때 학습한다는 차이
# 정해진 Epoch 만큼 반복하며, 데이터로더로부터 데이터를 받아 학습하고, 모델이 학습하고 예측한 결과물에 대해 손실함수를 적용하여 예측이 얼마나 틀어졌는지 확인
#-----------------------------------------------------------------------------------------------------------
                if i % 3000 == 0:
                    print(f"Epoch: {epoch}/{epochs}, Iteration: {i}, Loss: {running_loss / 3000}")
                    global_step = input_model.current_round * steps + epoch * len(train_loader) + i
                    summary_writer.add_scalar(tag="loss_for_each_batch", scalar=running_loss, global_step=global_step)
                    running_loss = 0.0
                  # 여긴 학습진행중일때 사용자에게 보여주는 출력문입니다

        print("Finished Training")
        # 여기도요
#-----------------------------------------------------------------------------------------------------------
        PATH = "./cifar_net.pth"
        torch.save(model.state_dict(), PATH)
        # pytorch의 모델 저장 파일방식인 pth 파일 경로를 지정하고 현재 model의 state dict를 뽑아 저장
        # 껍데기는 두고 알맹이(모델의 파라미터)만 복제해간다고 생각하면 좋겠습니다.

        output_model = flare.FLModel(
            params=model.cpu().state_dict(),
            meta={"NUM_STEPS_CURRENT_ROUND": steps},
        )
        # 최종 결과물 모델은 Flare 프레임워크의 FLmodel 함수를 불러와서
        # 모델파라미터는 지금저장된 model의 파라미터를 cpu에 올린다음 불러오고
        # 학습 진행이 얼마나 되었는지 step수를 저장함

        flare.send(output_model)
        # 결과물 모델은 Flare에 전송 (서버로)
        
# 여기까지 로컬에서 학습이 종료된 모델을 저장하고, 모델 파라미터를 뽑아 서버로 전송하여주는 부분임
#-----------------------------------------------------------------------------------------------------------

if __name__ == "__main__":
    main()

## NVflare POC

NVflare POC  
https://nvflare.readthedocs.io/en/2.5.1/user_guide/nvflare_cli/poc_command.html#poc-command  

Proof of Concept 모드는 시뮬레이션과 유사하게 하나의 단일 컴퓨터에서 연합학습 환경을 실험할 수 있습니다.  
시뮬레이터와 달리 서버 및 각 클라이언트를 서로 다른 프로세스로 표현하며, 이들의 실행환경을 설정하고 연결하는 프로비저닝 단계부터 진행합니다.  
연합학습을 위한 환경 구성과 클라이언트 연결, 프로젝트 배포 옵션을 실험 해 볼 수 있습니다.  

아래는 POC 모드의 명령어 몇가지입니다.

In [ ]:
nvflare poc prepare
# 서버와 참여클라이언트 수, 이름, 사용할 프로젝트 파일 등을 지정하여 작업 준비 / 기본 working 디렉토리인 /tmp/nvflare/poc 위치에 start up kit 생성

nvflare poc prepare-jobs-dir
# 사용할 job 을 제출합니다

nvflare poc start
# poc 모드 작동을 시작합니다

nvflare poc stop
# poc모드 작동을 정지합니다

nvflare poc clean
# poc working directory를 삭제합니다

In [ ]:
nvflare poc prepare
# prepare를 실행하면 스타트업 키트가 생성되며, 전체 연합학습 기능을 시작할 준비가 됩니다.

nvflare poc start
# start 명령은 서버, 클라이언트, 관리 콘솔까지 모든 인스턴스를 한번에 실행합니다

# -p 옵션을 통해 원하는 인스턴스만 먼저 따로 실행가능합니다.
# -p server, client, admin@nvidia.com(콘솔)

# -gpu 옵션을 통해 원하는 GPU 장치를 원하는 클라이언트가 사용하도록 연결 할 수 있습니다.
# client1, 2, 3, 4
# -gpu 2 1 0 3
# client1==gpu[2] / client2==gpu[1] / client3==gpu[0] / client4==gpu[3]

nvflare poc prepare-jobs-dir -j {NVFlare/examples}
# 원하는 job이 정의된 디렉토리를 관리자에게 연결하고 콘솔에서 제출 시 사용할 수 있습니다.
# nvflare poc start -p admin@nvidia.com / admin@nvidia.com라는 관리자 계정으로 콘솔 시작
# submit_job hello-world/hello-numpy-sag/jobs/hello-numpy-sag / 기존에 연결해준 NVFlare/examples 디렉토리 아래 job 경로

nvflare poc stop
# job을 제출하고 학습이 완료되면 stop을 사용하여 중지할 수 있습니다.
# 또는 -p 옵션으로 특정 인스턴스만 종료가 가능합니다.
# -p server / 서버만 종료

### job 이란?

NVflare Jobs  
https://nvflare.readthedocs.io/en/main/real_world_fl/job.html  

job은 관리자(사용자)가 진행할 실험에 대한 모든 정보입니다.  
따라서 서버와 클라이언트의 연결 정보, 필요한 리소스, 실험 전략, app등이 모두 포함된 일종의 작업계획서라고 생각할 수 있겠습니다.  
(여기서의 app은 서버 및 클라이언트의 실행 정보입니다. 서버가 사용할 flare 버전이나 송수신되는 데이터 필터링같은 내용들을 포함합니다.)